In [1]:
!pip install tensorflow librosa seaborn scikit-learn tensorflow_datasets


  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached tensorflow_datasets-4.9.8-py3-none-any.whl.metadata (11 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached dm_tree-0.1.9-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached simple_parsing-0.1.7-py3-none-any.whl.metadata (7.3 kB)
Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl (376.0 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached tensorflow_datasets-4.9.8-py3-none-any.whl (5.3 MB)
Using cached keras-3.9.2-py3-none-any.whl (1.3 MB)
Using cached dm_tree-0.1.9-cp312-cp312-win_amd64.whl (102 kB)
Using cached simple_parsing-0.1.7-py3-none-any.whl (112 kB)


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\import_pb_to_tensorboard.exe' -> 'C:\\Python312\\Scripts\\import_pb_to_tensorboard.exe.deleteme'


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install librosa



  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl.metadata (16 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
Using cached librosa-0.11.0-py3-none-any.whl (260 kB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl (1.0 MB)

   -------- ------------------------------- 1/5 [audioread]
   ---------------- ----------------------- 2/5 [soundfile]
   ---------------- ----------------------- 2/5 [soundfile]
   ------------------------ --------------- 3/5 [pooch]
   ------------------------ --------------- 3/5 [pooch]
   ------------------------ --------------- 3/5 [pooch]
   -------------------------------- ------- 4/5 [librosa]
   -------------------------------- ------- 4/5 [librosa]
   -------------------------------- ------

In [7]:
pip install pydub

In [ ]:
import tensorflow as tf
import numpy as np
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

!pip install tensorflow_datasets
import tensorflow_datasets as tfds

SELECTED_CLASSES = ["yes", "no", "stop", "go", "right"]

dataset, info = tfds.load(
    "speech_commands",
    split="train",
    as_supervised=True,
    with_info=True
)

label_names = info.features["label"].names
class_indices = [label_names.index(word) for word in SELECTED_CLASSES]

def extract_mfcc(audio, sample_rate=16000, max_pad_len=40):
    audio = audio.astype(np.float32)
    mfccs = librosa.feature.mfcc(y=audio.flatten(), sr=sample_rate, n_mfcc=40)
    pad_w = max_pad_len - mfccs.shape[1]
    if pad_w > 0:
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_w)), mode="constant")
    else:
        mfccs = mfccs[:, :max_pad_len]
    return mfccs.flatten()

x, y = [], []

for audio, label in tfds.as_numpy(dataset.take(10000)):
    if label in class_indices:
        x.append(extract_mfcc(audio))
        y.append(class_indices.index(label))

x = np.array(x, dtype=np.float32)
y = np.array(y, dtype=np.int64)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation="relu", input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(5, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=(x_test, y_test)
)

model.save("speech_command_model.h5")

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"\nTest Accuracy: {test_acc:.2f}")

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=SELECTED_CLASSES))

conf_matrix = confusion_matrix(y_test, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=SELECTED_CLASSES,
    yticklabels=SELECTED_CLASSES
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()